In [2]:
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from keras import ops
from keras.layers import StringLookup
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
#os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

In [3]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

In [4]:
gpu_devices = tf.config.list_physical_devices('GPU')
if not gpu_devices:
    print("TensorFlow is using the CPU.")
else:
    print(f"TensorFlow is using the following GPU(s): {gpu_devices}")

TensorFlow is using the following GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
dataPath = "/home/b1az3/code/ml/trainingData/" #FILEPATH


--- words.txt ---------------------------------------------------------------#

 iam database word information

 format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A

     a01-000u-00-00  -> word id for line 00 in form a01-000u
     ok              -> result of word segmentation
                            ok: word was correctly
                            er: segmentation of word can be bad

     154             -> graylevel to binarize the line containing this word
     1               -> number of components for this word NOTE: dont think this exists idk why
     408 768 27 51   -> bounding box around this word in x,y,w,h format
     AT              -> the grammatical tag for this word, see the
                        file tagset.txt for an explanation
     A               -> the transcription for this word


In [6]:
text = open(dataPath + "ascii/words.txt","r").read().splitlines()
text = text[18:] #first 18 lines is metadata

images = []
for data in text:
    parts = data.split(" ")
    word = parts[-1]
    tmp = parts[0].split("-")
    path = dataPath + "final_aug_words/IAM/" + tmp[0]+"/"+tmp[0]+"-"+tmp[1]+"/"+parts[0]+"/"
    if len(word) == 1:
        inDir = os.listdir(path)
        for f in inDir:
            images.append([os.path.join(path+f), word]) #last 2 are width and height ord() to convert to ascii

def recurSearch (path,letter):
    files = os.listdir(path)
    if files[0][:5] == "image":
        for f in files:
            images.append([os.path.join(path,f), letter])
    else:
        for f in files:
            recurSearch(os.path.join(path,f), letter)

letters = os.listdir(dataPath + "final_aug_words/EMNIST/")
print (letters)
print ([chr(int(ltr,16)) for ltr in letters])
for ltr in letters: #ltr will be in hex (gives ascii value)
    path = dataPath + "final_aug_words/EMNIST/"+ltr+"/"
    intLtr = chr(int(ltr,16))
    recurSearch(path,intLtr)

images = np.array(images)
np.random.shuffle(images)

print (images[0:10])
    
#2 mins

['73', '5a', '47', '64', '55', '74', '53', '59', '78', '7a', '54', '79', '63', '56', '45', '4e', '6c', '35', '52', '51', '6f', '38', '50', '46', '58', '61', '39', '37', '30', '43', '4c', '70', '72', '34', '48', '67', '6e', '6a', '33', '68', '4f', '6d', '4b', '77', '42', '32', '4d', '49', '76', '57', '65', '62', '31', '71', '4a', '6b', '75', '41', '69', '36', '66', '44']
['s', 'Z', 'G', 'd', 'U', 't', 'S', 'Y', 'x', 'z', 'T', 'y', 'c', 'V', 'E', 'N', 'l', '5', 'R', 'Q', 'o', '8', 'P', 'F', 'X', 'a', '9', '7', '0', 'C', 'L', 'p', 'r', '4', 'H', 'g', 'n', 'j', '3', 'h', 'O', 'm', 'K', 'w', 'B', '2', 'M', 'I', 'v', 'W', 'e', 'b', '1', 'q', 'J', 'k', 'u', 'A', 'i', '6', 'f', 'D']
[['/home/b1az3/code/ml/trainingData/final_aug_words/EMNIST/37/train_37/train_37_15174/image_0_8737'
  '7']
 ['/home/b1az3/code/ml/trainingData/final_aug_words/EMNIST/59/hsf_1/hsf_1_00544/image_0_1770'
  'Y']
 ['/home/b1az3/code/ml/trainingData/final_aug_words/EMNIST/4c/train_4c/train_4c_02279/image_0_5036'
  'L']
 

In [ ]:
#TOO MUCH DATA here to shorten it so it doesnt take forever for testing
images = images[:1000000]

In [8]:
trainRatio = 0.8
valRatio = 0.15
trainImages = images[0:int(len(images) * trainRatio)]
valImages = images[int(len(images) * trainRatio):int(len(images) * (trainRatio + valRatio))]
testImages = images[int(len(images) * (trainRatio + valRatio)):]

In [9]:
unique_labels = sorted(set(images[:, 1]))  # Unique characters
num_classes = len(unique_labels)

In [10]:
label_encoder = tf.keras.layers.StringLookup(vocabulary=unique_labels, num_oov_indices=0)
def load_image(image_path, label):
    image = tf.io.read_file(image_path)  # Read image
    image = tf.image.decode_jpeg(image, channels=3)  # Decode as RGB
    image = tf.image.resize(image, (224, 224))  # Resize to match input shape of VGG16
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize to [0, 1]
    image = preprocess_input(image)  # Preprocess input for VGG16d
    label = label_encoder(label)
    return image, label
print(load_image(images[0,0],images[0,1]))


(<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 151.061  ,  138.22101,  131.32   ],
        [ 151.061  ,  138.22101,  131.32   ],
        [ 151.061  ,  138.22101,  131.32   ],
        ...,
        [ 151.061  ,  138.22101,  131.32   ],
        [ 148.061  ,  135.22101,  128.32   ],
        [-102.939  , -115.779  , -122.68   ]],

       [[ 151.061  ,  138.22101,  131.32   ],
        [ 151.061  ,  138.22101,  131.32   ],
        [ 151.061  ,  138.22101,  131.32   ],
        ...,
        [ 151.061  ,  138.22101,  131.32   ],
        [ 148.061  ,  135.22101,  128.32   ],
        [-102.939  , -115.779  , -122.68   ]],

       [[ 151.061  ,  138.22101,  131.32   ],
        [ 151.061  ,  138.22101,  131.32   ],
        [ 151.061  ,  138.22101,  131.32   ],
        ...,
        [ 151.061  ,  138.22101,  131.32   ],
        [ 148.061  ,  135.22101,  128.32   ],
        [-102.939  , -115.779  , -122.68   ]],

       ...,

       [[ 149.061  ,  136.22101,  129.32   ],
        [ 14

I0000 00:00:1741856453.245536   21238 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9623 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [11]:
train_ds = tf.data.Dataset.from_tensor_slices((list(trainImages[:,0]), list(trainImages[:,1])))
val_ds = tf.data.Dataset.from_tensor_slices((list(valImages[:,0]), list(valImages[:,1])))
test_ds = tf.data.Dataset.from_tensor_slices((list(testImages[:,0]), list(testImages[:,1])))

In [12]:
BATCH_SIZE = 16
train_ds = train_ds.map(load_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(load_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.summary()
# Freeze base model (optional, for feature extraction)
base_model.trainable = False  

# Add custom classifier
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dense(4096, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Multi-class classification
])

# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 75)             │       307,275 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,567,819 (513.34 MB)

 Trainable params: 119,853,131 (457.20 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Train the model
EPOCHS = 6  # Change as needed

model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

Epoch 1/6


2025-03-13 17:02:47.799277: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.39 = (f32[16,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,3,224,224]{3,2,1,0} %bitcast.1408, f32[64,3,3,3]{3,2,1,0} %bitcast.1415, f32[64]{0} %bitcast.1417), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1_1/vgg16_1/block1_conv1_1/convolution" source_file="/home/b1az3/anaconda3/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-03-13 17:02:48.057691: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549]

49998/50000 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6483 - loss: 1.3289